In [11]:
from PIL import Image, ImageDraw
import face_recognition
from IPython import display
import numpy as np
import os, sys
import matplotlib.pyplot as plt
import seaborn as sns
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import multiprocessing
import scipy.io.wavfile as wav
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.recurrent import LSTM
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
sns.set_palette("colorblind")
sns.set_context("paper", font_scale=2)
sns.set_style("white")
%matplotlib inline


```bash
find . -name "20*.mp4" | parallel --no-notice ffmpeg-normalize -v {}
```
```bash
find . -name "20*.mp4" | parallel --no-notice ffmpeg -nostdin -i {} {.}.gif -hide_banner
```

In [59]:
datadir = "../obama"
savedir = datadir
mp4files = !find $datadir -name "20*.mp4"
fileIDs = [os.path.splitext(os.path.basename(mp4file))[0] for mp4file in mp4files]
fileIDs

['20161231_Weekly_Address_HD',
 '20160813_Weekly_Address_HD',
 '20160625_Weekly_Address_HD',
 '20170114_Weekly_Address_HD',
 '20161022_Weekly_Address_HD',
 '20161001_Weekly_Address_HD',
 '20161105_Weekly_Address_HD',
 '20161210_Weekly_Address_HD',
 '20160910_Weekly_Address_HD',
 '20160903_Weekly_Address_HD',
 '20160716_Weekly_Address_HD']

In [40]:
# Split data for training/validation
tID = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
vID = [2]
tfileIDs = [fileIDs[id_] for id_ in tID]
vfileIDs = [fileIDs[id_] for id_ in vID]
print("For training {}, \n for validation {}".format(tfileIDs, vfileIDs))

For training ['20161231_Weekly_Address_HD', '20160813_Weekly_Address_HD', '20160625_Weekly_Address_HD', '20170114_Weekly_Address_HD', '20161022_Weekly_Address_HD', '20161001_Weekly_Address_HD', '20161105_Weekly_Address_HD', '20161210_Weekly_Address_HD', '20160910_Weekly_Address_HD', '20160903_Weekly_Address_HD'], 
 for validation ['20160625_Weekly_Address_HD']


In [42]:
gif_files = [id_+".gif" for id_ in fileIDs]
tgif_files = [gif_files[id_] for id_ in tID]
vgif_files = [gif_files[id_] for id_ in vID]
wav_files = ["normalized-"+id_+".wav" for id_ in fileIDs]
twav_files = [wav_files[id_] for id_ in tID]
vwav_files = [wav_files[id_] for id_ in vID]

In [52]:
def get_giffile_from_ID(fileID):
  return fileID+".gif"
def get_wavfile_from_ID(fileID):
  return "normalized-"+fileID+".wav"

## Generate lip features

In [65]:
def get_lipfeatures_from_im(im):
  # Find all facial features in all the faces in the image
  face_landmarks_list = face_recognition.face_landmarks(im)
  facial_features = [
    'top_lip',
    'bottom_lip']
  lip_features_list = list()
  for face_landmarks in face_landmarks_list:
    # Print the location of each facial feature in this image
    for facial_feature in facial_features:
      for x, y in face_landmarks[facial_feature]:
        lip_features_list.append([x, y])
  return np.array(lip_features_list)

In [66]:
def generate_lipfeature_from_ID(fileID):
  giffile = os.path.join(datadir, get_giffile_from_ID(fileID))
  wavfile = os.path.join(datadir, get_wavfile_from_ID(fileID))
  gif_im = Image.open(giffile)
  gif_im_array = np.asarray(gif_im)
  gif_im_array.flags.writeable = True
  (rate,sig) = wav.read(wavfile)
  duration_s = sig.shape[0] / rate
  sec_per_frame_s = duration_s / gif_im.n_frames
  # cal value for t = 0
  lip_features = get_lipfeatures_from_im(gif_im_array)
  time_stamps = list()
  t_s = 0
  time_stamps.append(t_s)
  gif_lip_features_list = list()
  # cal value for t > 0
  for frame in range(1, gif_im.n_frames):
    gif_im.seek(gif_im.tell() + 1)
    gif_im_array = np.asarray(gif_im)
    gif_im_array.flags.writeable = True
    lip_features = get_lipfeatures_from_im(gif_im_array)
    if len(lip_features) > 1:
      gif_lip_features_list.append(lip_features)
      time_stamps.append(t_s)
    t_s += sec_per_frame_s
  return time_stamps, np.array(gif_lip_features_list)
def map_generate_lipfeature(fileID):
  time_stamps, gif_lip_features_mat = generate_lipfeature_from_ID(fileID)
  featfile = os.path.join(savedir, "giffeat_"+fileID+".npy")
  tsfile = os.path.join(savedir, "giftime_"+fileID+".npy")
  np.save(featfile, gif_lip_features_mat)
  np.save(tsfile, time_stamps)

In [67]:
# It's take long time! Be aware to execute.
if False:
  nthreads = len(fileIDs)
  pool = multiprocessing.Pool(nthreads)
  pool.map(map_generate_lipfeature, fileIDs)
  pool.close()

FileNotFoundError: [Errno 2] No such file or directory: '../obama/normalized-20160625_Weekly_Address_HD.wav'

## Generate auido features